In [1]:
from mode import *

In [2]:
keys = [np.random.randint(0, 2**64, dtype=np.uint64) for _ in range(2)]
print("Keys:")
print(f"{keys[0]:x}")
print(f"{keys[1]:x}")


Keys:
938c9dd395be88d4
105a7e10c630fd79


In [59]:
message = int(''.join([str(np.random.randint(0, 2**64, dtype=np.uint64)) for _ in range(16)]))
print("Message:", f"{message:x}")

aux, cipher, tag = encrypt_and_authenticate(keys, message)
print("Cipher:", f"{cipher:x}")


verified, decrypted = decrypt_and_verify(keys, cipher, tag, aux)
print("Verified:", verified)
if verified:
    print("Decrypted:", f"{decrypted:x}")


Message: 1ade819fa367bd2df842678c59618504751bc091d9156cd6b4e00153ca21bae801a620424867db493f26f5cb8305e85d221b6bd61ba688455182590aff4d45b0b2d349d49906f78afe577542549c4bd46cee8a8ca4b6029f574e94f090d319e7ad61cafd0581074bcad6117c9953fe1f55b3d00a0995c046ee0ef923b183ab55d0
Cipher: 14fcffc34b72e79542af01071717caa23b0165ac9141eab1730b216f68244e2cdc3d94a53d8e5aff0cd6e83b2e229a2cd67aae52a89eb14d092184b1f38de19174bf249aaa57bdfa61956e967d2dcb75264f3938c875a2f1ad924a848b6bd1ae485354697c625d778a5b5051b16a4b5b2826f8d7e6f0a641446f52a9fbc6aea3cbcdffc809ff3452
Verified: True
Decrypted: 1ade819fa367bd2df842678c59618504751bc091d9156cd6b4e00153ca21bae801a620424867db493f26f5cb8305e85d221b6bd61ba688455182590aff4d45b0b2d349d49906f78afe577542549c4bd46cee8a8ca4b6029f574e94f090d319e7ad61cafd0581074bcad6117c9953fe1f55b3d00a0995c046ee0ef923b183ab55d0


In [4]:
from hash16 import hash

a = message
b = message + 2**16

print(f"{hash(a):x}")
print(f"{hash(b):x}")

e9c241aa765b61aa0b36805fa3ae27cf
e9c241aa765b61aa0b36805f73dd27cf


In [5]:
import random
from hash16 import *  

def random_walk_hash(start, steps=1000):
    current = start
    print(f"Départ : {hex(current)}")
    for i in range(steps):
        # Mar​che aléatoire : flip 1 bit aléatoire
        bit_to_flip = 1 << random.randint(0, 127)
        current ^= bit_to_flip
        h = hash(current)
        print(f"Étape {i+1:4d} : message = {hex(current)} | hash = {hex(h)}")

start_msg = random.getrandbits(128)
random_walk_hash(start_msg, steps=50)


Départ : 0x886009144c665705a1b428e52bf8d2ac
Étape    1 : message = 0x886009144c665605a1b428e52bf8d2ac | hash = 0x14d70a545a774812495fdc0ca44ffc31
Étape    2 : message = 0x886009144c665405a1b428e52bf8d2ac | hash = 0x14d70a545a774872495fdc0ca44ffc31
Étape    3 : message = 0x886009144c665405a1b420e52bf8d2ac | hash = 0x14d70a545a774872495f69a3a44ffc31
Étape    4 : message = 0x886009144c665405a1b420e52bf8c2ac | hash = 0x14d70a545a774872495f69a3a44f552b
Étape    5 : message = 0x886009164c665405a1b420e52bf8c2ac | hash = 0x14d76a7a5a774872495f69a3a44f552b
Étape    6 : message = 0x886009164c665405a5b420e52bf8c2ac | hash = 0x14d76a7a5a774872491f69a3a44f552b
Étape    7 : message = 0x886009164c665405a5f420e52bf8c2ac | hash = 0x14d76a7a5a774872494369a3a44f552b
Étape    8 : message = 0x886009164c665405a5f420e52bf842ac | hash = 0x14d76a7a5a774872494369a3a44ffd50
Étape    9 : message = 0x886009164c665425a5f420e52bf842ac | hash = 0x14d76a7a5a774834494369a3a44ffd50
Étape   10 : message = 0x886009164c665

In [ ]:
from matrix_ops import *

matrix = circular_matrix_from_vector(np.uint64(0xc063fe883bca8005))
matrix_inv = matrix_inverse(matrix)

print("MATRIX:")
for line in matrix:
    print(f"{line:x}")

print("INVERSE:")
for line in matrix_inv:
    print(f"{line:x}")